In [14]:
import pandas as pd
from flask import Flask, render_template
from bs4 import BeautifulSoup as bs
import requests
import re
from splinter import Browser

In [3]:
app = Flask(__name__)

# Get html tables with pandas
# url = 'https://space-facts.com/mars/'
# tables = pd.read_html(url, header=0)
# tables

In [8]:
res = "https://space-facts.com/mars/"

mars_facts = pd.read_html(res)

mars_facts_df = mars_facts[0]
mars_facts_df
# soup = bs(res.content,'lxml')
# table = soup.find_all('table')

# mars_fact = df[0].to_json(orient='records')
# print(mars_fact)


,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [9]:
mars_html = mars_facts_df.to_html(index= False, header= None)
mars_html

'<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

In [10]:
# @app.route("/")
# def index():
#     return render_template("index.html", mars_fact=mars_fact)


# if __name__ == "__main__":
#     app.run(debug=True)

In [12]:
mars_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
response = requests.get(mars_url)
mars_soup = bs(response.text, 'html.parser')

In [17]:
browser = Browser('chrome', executable_path='chromedriver', headless=False)
browser.visit(mars_url)

In [19]:
mars_img_list = mars_hem_soup.find_all('a', class_='itemLink')
hemisphere_image_urls = []

for hemisphere_img in mars_img_list:
    
    hemisphere_dict = {}
    
    title = hemisphere_img.h3.text
    
    browser.click_link_by_partial_text(title)
    
    image_url = browser.find_by_css('.wide-image').first['src']
    
    hemisphere_dict['title'] = title
    hemisphere_dict['img_url'] = image_url
    
    hemisphere_image_urls.append(hemisphere_dict) 
    
    browser.back()
    

hemisphere_image_urls

[{'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [20]:
mars_twitter = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(mars_twitter)

mars_tweet_soup = bs(response.text, "html.parser")
mars_tweets = mars_tweet_soup.find_all('div', class_='tweet')

i = 0

for tweet in mars_tweets:
    
    tweet_timestamp = int(tweet.find('span', class_= '_timestamp')['data-time'])
    
    tweet_text = tweet.find('p', class_ = 'tweet-text').text
    
    if (re.match(r'Sol \d\d\d\d \(', tweet_text) is not None) and (tweet_timestamp > i):
        
        mars_weather = tweet_text
        
        i = tweet_timestamp
        
mars_weather

'Sol 1951 (Jan 31, 2018), Sunny, high -23C/-9F, low -77C/-106F, pressure at 7.49 hPa, daylight 05:42-17:28'